# Red Wine Quality
`redwine-quality.ipynb`

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('./winequality-red.csv')

In [4]:
# 특성 살펴보기 (품질 분포, 히스토그램)

In [ ]:
# 품질과의 상관관계 (히트맵) / 품질에 영향을 많이 주는 특성들 시각화

In [8]:
# 훈련 테스트 셋 나누기

In [ ]:
# KNN 회귀로 와인 품질 예측 

In [ ]:
# 선형회귀로 와인 품질 예측 (훈련셋/테스트셋의 점수를 높여보자)

In [11]:
# Ridge, Lasso 로 규제해가며 예측 